<a href="https://colab.research.google.com/github/amrutadeo-22/resnet/blob/main/AggregatedResnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

import os


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, shortcut_type='B'):
        super(Bottleneck, self).__init__()
        self.stride = stride

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        if shortcut_type == 'C' or (shortcut_type == 'B' and in_channels != out_channels * self.expansion):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )
        elif in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.AvgPool2d(1, stride=stride),
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        out = nn.ReLU(inplace=True)(self.bn1(self.conv1(x)))
        out = nn.ReLU(inplace=True)(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = nn.ReLU(inplace=True)(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = nn.ReLU(inplace=True)(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AvgPool2d(4)(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet50(num_classes=10):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes)


def train(epoch, net, trainloader, optimizer, criterion, device):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(inputs)}/{len(trainloader.dataset)}] '
                  f'Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}%')

def test(epoch, net, testloader, criterion, device):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'Test Epoch: {epoch} | Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}%')


def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'


    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)


    net = ResNet50(num_classes=10).to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)


    for epoch in range(0, 200):
        train(epoch, net, trainloader, optimizer, criterion, device)
        test(epoch, net, testloader, criterion, device)
        scheduler.step()

if __name__ == '__main__':
    main()


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified
Train Epoch: 0 [0/50000] Loss: 2.449 | Acc: 6.250%
Train Epoch: 0 [1280/50000] Loss: 14.219 | Acc: 10.369%
Train Epoch: 0 [2560/50000] Loss: 9.148 | Acc: 10.938%
Train Epoch: 0 [3840/50000] Loss: 7.365 | Acc: 10.811%
Train Epoch: 0 [5120/50000] Loss: 6.243 | Acc: 10.518%
Train Epoch: 0 [6400/50000] Loss: 5.497 | Acc: 10.570%
Train Epoch: 0 [7680/50000] Loss: 5.025 | Acc: 10.502%
Train Epoch: 0 [8960/50000] Loss: 4.652 | Acc: 10.618%
Train Epoch: 0 [10240/50000] Loss: 4.368 | Acc: 10.909%
Train Epoch: 0 [11520/50000] Loss: 4.139 | Acc: 11.461%
Train Epoch: 0 [12800/50000] Loss: 3.991 | Acc: 11.402%
Train Epoch: 0 [14080/50000] Loss: 3.849 | Acc: 11.423%
Train Epoch: 0 [15360/50000] Loss: 3.733 | Acc: 11.312%
Train Epoch: 0 [16640/50000] Loss: 3.630 | Acc: 11.367%
Train Epoch: 0 [17920/50000] Loss: 3.540 | Acc: 11.242%
Train Epoch: 0 [19200/50000] Loss: 3.463 | Acc: 11.222%
Train Epoch: 0 [20480/50000] Loss: 3.393 | Acc: 11.432%
Train Epoch: 0 [21760

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CIFAR-10 transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download CIFAR-10
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Custom ResNeXt Block
class ResNeXtBlock(nn.Module):
    def __init__(self, in_channels, cardinality, bottleneck_width, stride=1):
        super().__init__()
        D = cardinality * bottleneck_width
        self.conv_reduce = nn.Conv2d(in_channels, D, kernel_size=1, bias=False)
        self.bn_reduce = nn.BatchNorm2d(D)
        self.conv_conv = nn.Conv2d(D, D, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.bn = nn.BatchNorm2d(D)
        self.conv_expand = nn.Conv2d(D, in_channels, kernel_size=1, bias=False)
        self.bn_expand = nn.BatchNorm2d(in_channels)

        self.shortcut = nn.Sequential()
        if stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, in_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(in_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn_reduce(self.conv_reduce(x)))
        out = F.relu(self.bn(self.conv_conv(out)))
        out = self.bn_expand(self.conv_expand(out))
        out += self.shortcut(x)
        return F.relu(out)

# ResNeXt for CIFAR-10
class ResNeXt(nn.Module):
    def __init__(self, num_classes=10, blocks=3):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, blocks, 32, 4)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, in_channels, blocks, cardinality, bottleneck_width):
        layers = []
        for _ in range(blocks):
            layers.append(ResNeXtBlock(in_channels, cardinality, bottleneck_width))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# Training function
def train(model, loader, optimizer, criterion):
    model.train()
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Run training and evaluation
def run_experiment(train_ratio):
    length = len(dataset)
    train_len = int(train_ratio * length)
    test_len = length - train_len
    train_data, test_data = random_split(dataset, [train_len, test_len])

    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=64)

    model = ResNeXt().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(10):
        train(model, train_loader, optimizer, criterion)

    accuracy = evaluate(model, test_loader)
    print(f"Train Ratio: {int(train_ratio*100)}% - Test Accuracy: {accuracy:.2f}%")

# Run all splits
for ratio in [0.9, 0.8, 0.7, 0.6]:
    run_experiment(ratio)
